In [83]:
from doc2vec import *




In [84]:
gloves = load_glove('glove.6B.300d.txt')

In [85]:
def load_articles(articles_dirname, gloves):
    """
    Load all .txt files under articles_dirname and return a table (list of lists/tuples)
    where each record is a list of:
      [filename, title, article-text-minus-title, wordvec-centroid-for-article-text]
    We use gloves parameter to compute the word vectors and centroid.
    The filename is stripped of the prefix of the articles_dirname pulled in as
    script parameter sys.argv[2]. E.g., filename will be "business/223.txt"
    """
    flist = filelist(articles_dirname)
    

In [86]:
flist = filelist('bbc')

In [87]:
gloves['the']

array([  4.65600000e-02,   2.13180000e-01,  -7.43640000e-03,
        -4.58540000e-01,  -3.56390000e-02,   2.36430000e-01,
        -2.88360000e-01,   2.15210000e-01,  -1.34860000e-01,
        -1.64130000e+00,  -2.60910000e-01,   3.24340000e-02,
         5.66210000e-02,  -4.32960000e-02,  -2.16720000e-02,
         2.24760000e-01,  -7.51290000e-02,  -6.70180000e-02,
        -1.42470000e-01,   3.88250000e-02,  -1.89510000e-01,
         2.99770000e-01,   3.93050000e-01,   1.78870000e-01,
        -1.73430000e-01,  -2.11780000e-01,   2.36170000e-01,
        -6.36810000e-02,  -4.23180000e-01,  -1.16610000e-01,
         9.37540000e-02,   1.72960000e-01,  -3.30730000e-01,
         4.91120000e-01,  -6.89950000e-01,  -9.24620000e-02,
         2.47420000e-01,  -1.79910000e-01,   9.79080000e-02,
         8.31180000e-02,   1.52990000e-01,  -2.72760000e-01,
        -3.89340000e-02,   5.44530000e-01,   5.37370000e-01,
         2.91050000e-01,  -7.35140000e-03,   4.78800000e-02,
        -4.07600000e-01,

In [88]:
ENGLISH_STOP_WORDS = frozenset([
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"])

In [89]:
import nltk
import re
def words(text):
    """
    Given a string, return a list of words normalized as follows.
    Split the string to make words first by using regex compile() function
    and string.punctuation + '0-9\\r\\t\\n]' to replace all those
    char with a space character.
    Split on space to get word list.
    Ignore words < 3 char long.
    Lowercase all words
    Remove English stop words
    """
    text = text.lower()
    text = re.sub('[' + string.punctuation + '0-9\\r\\t\\n]', ' ', text)
    text = text.split(" ")
    text = [w for w in text if len(w) > 3]
    text = [w for w in text if not w in ENGLISH_STOP_WORDS]
    return text

def doc2vec(text, gloves):
    """
    Return the word vector centroid for the text. Sum the word vectors
    for each word and then divide by the number of words. Ignore words
    not in gloves.
    """
    wlist = words(text)
    sum = np.zeros(shape=(300,))
    for word in wlist:
        if word in gloves:
            sum = sum + gloves[word]
    centroid = sum / len(wlist)
    return centroid

In [94]:
articles = []
for file in flist:
    text = get_text(file)
    lines = text.split('\n')
    title = lines[0]
    contents = lines[1:]
    contents = " ".join(contents)
    centroid = doc2vec(contents, glove)    
    names = file.split('/')
    if len(names) == 3:
        filename = names[1]+"/" + names[2]
    articles.append((filename, title, contents, centroid))
    

In [95]:
np.shape(articles[1][3])

(300,)

In [75]:
sum = np.zeros(shape=(300,))
sum = sum + gloves['the']
np.array_equal(gloves['the'], sum)

True

In [73]:
np.shape(sum)

(1, 300)

In [74]:
np.shape(gloves['the'])

(300,)

In [109]:
a = [(1,2), (3,3), (2,4)]
b = sorted(a, key=lambda x: x[0])

In [112]:
for x in a:
    print x[1]

2
3
4
